<a href="https://colab.research.google.com/github/hemanthkumar17/minGPT/blob/master/Pile_tests.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import torch
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
import random

In [ ]:
!git clone https://git@github.com/hemanthkumar17/minGPT.git

Cloning into 'minGPT'...
remote: Enumerating objects: 405, done.
remote: Counting objects: 100% (8/8), done.
remote: Compressing objects: 100% (6/6), done.
remote: Total 405 (delta 2), reused 6 (delta 2), pack-reused 397
Receiving objects: 100% (405/405), 1.43 MiB | 4.28 MiB/s, done.
Resolving deltas: 100% (198/198), done.


In [ ]:
!pip install -q transformers datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 58.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.7/468.7 KB 41.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 67.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 KB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 KB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 61.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 KB 15.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 KB 25.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 KB 19.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 KB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 KB 29.5 MB/s eta 0:00:00


In [ ]:
from datasets import load_dataset


In [ ]:
%cd minGPT

/content/minGPT


In [ ]:
from mingpt.model import GPT

model_config = GPT.get_default_config()
model_config.model_type = 'gpt-nano'
model_config.vocab_size = 50304
model_config.block_size = 20
model = GPT(model_config)

number of parameters: 2.59M


In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("EleutherAI/pythia-70m-deduped")

# model = AutoModelForCausalLM.from_pretrained("EleutherAI/pythia-70m-deduped")

In [ ]:
import torch.utils.data as data
import torch.nn as nn
from datasets import load_dataset
from torch.utils.data import DataLoader, WeightedRandomSampler
from datasets import Dataset

seed = torch.Generator().manual_seed(42)


def padding(s):
    maxlen=20
    if len(s['input_ids'])<maxlen :
        s['input_ids']=s['input_ids']+[0]*(maxlen-len(s['input_ids']))
    return s

train_set = load_dataset("EleutherAI/the_pile", "enron_emails", split="train[50%:55%]")
train_set = train_set.map(lambda dataset: tokenizer(dataset["text"],truncation=True, max_length=20)).map(padding)
train_set= Dataset.from_dict({"id": train_set['input_ids']}).with_format("torch")        

train_set_size = int(len(train_set) * (1 - 0.2))
valid_set_size = len(train_set) - train_set_size
train_set, valid_set = data.random_split(train_set, [train_set_size, valid_set_size], generator = seed)

Generating train split: 0 examples [00:00, ? examples/s]

Dataset the_pile downloaded and prepared to /root/.cache/huggingface/datasets/EleutherAI___the_pile/enron_emails/0.0.0/6fadc480ecb32470826cbf5900a9558b791ce55d5e9a0fdc8ad653e7b64bb349. Subsequent calls will reuse this data.


Map:   0%|          | 0/25871 [00:00<?, ? examples/s]

Map:   0%|          | 0/25871 [00:00<?, ? examples/s]

In [ ]:
class Dict2Class(object):
      
    def __init__(self, my_dict):
          
        for key in my_dict:
            setattr(self, key, my_dict[key])

config = dict(
    MaxLen = 20,
    saveFolder = ".",
    epochs = 1,
    device = "cuda",
    vocab_size = 50304
)
config = Dict2Class(config)

In [ ]:
class TextGenerator(nn.Module):
    """A callback to generate text from a trained model.
    1. Feed some starting prompt to the model
    2. Predict probabilities for the next token
    3. Sample the next token and add it to the next input

    Arguments:
        max_tokens: Integer, the number of tokens to be generated after prompt.
        start_tokens: List of integers, the token indices for the starting prompt.
        index_to_word: List of strings, obtained from the TextVectorization layer.
        top_k: Integer, sample from the `top_k` token predictions.
        print_every: Integer, print after this many epochs.
    """

    def __init__(
        self, max_tokens, start_tokens, model, top_k=10, print_every=1, config = config
    ):
        super(TextGenerator, self).__init__()
        self.max_tokens = max_tokens
        self.start_tokens = start_tokens
#         self.index_to_word = index_to_word
        self.print_every = print_every
        self.k = top_k
        self.model=model
        self.config = config

    def sample_from(self, logits):
        logits, indices = torch.topk(logits, k=self.k, sorted=True)
        logits=logits.cpu()
        indices=indices.cpu()
        indices = np.asarray(indices).astype("int32")
       
        softmax=nn.Softmax(dim=0)
        preds = softmax(logits)
        preds = np.asarray(preds).astype("float32")
#         return np.random.choice(indices, p=preds) THIS IS THE CORRECT CODE, BUT HAD TO COMMENT IT AS
#.        PROBABILITIES HAVE NAN AND I HAD TO VERIFY PIPELINE, BELOW LINE WILL BE REMOVED ONCE NAN ISSUE 
#.        IS RESOLVED
        return np.random.choice(indices, p=preds)
#         return np.random.choice(5, 1, p=[0.1, 0, 0.3, 0.6, 0])

    def detokenize(self, number):
        return tokenizer.decode(number)

    def on_epoch_end(self, epoch, logs=None):
        start_tokens = [_ for _ in self.start_tokens]
        if (epoch + 1) % self.print_every != 0:
            return
        num_tokens_generated = 0
        tokens_generated = []
        attention_scores=[]
        while num_tokens_generated <= self.max_tokens:
            pad_len = self.config.maxlen - len(start_tokens)
            sample_index = len(start_tokens) - 1
            if pad_len < 0:
                data = start_tokens[:self.config.maxlen]
                sample_index = self.config.maxlen - 1
            elif pad_len > 0:
                data = start_tokens + [0] * pad_len
            else:
                data = start_tokens
                
            data = torch.Tensor(np.array([data])).type(torch.int32).to(config.device)
            
            y,attention_scores = self.model(data)
            sample_token = self.sample_from(y[0][sample_index])
            tokens_generated.append(sample_token)
            start_tokens.append(sample_token)
            num_tokens_generated = len(tokens_generated)
        txt = " ".join(
            [self.detokenize(_) for _ in self.start_tokens + tokens_generated]
        )
        print(f"generated text:\n{txt}\n")
        return attention_scores[0]

In [ ]:
def training(config=None, model = None):
    # with wandb.init(config=config) as run:
        # config = wandb.config
#         run.name = "Heads-"+str(config.Heads)+"Embedding-"+str(config.Embedding)+"MaxLen-"+str(config.MaxLen)
        # run.name="2 layers"
        device = "cuda"
        def padding(s):
            maxlen=config.MaxLen
            if len(s['input_ids'])<maxlen :
                s['input_ids']=s['input_ids']+[0]*(maxlen-len(s['input_ids']))
            return s
               
        # model=Model(config.Heads,config.Embedding,config.MaxLen)
        # model.load_state_dict(torch.load("/kaggle/input/weights2/transformer_weights.pth")) 
        model.to(device)
        
        # train_set = train_set.map(lambda dataset: tokenizer(dataset["text"],truncation=True, max_length=config.MaxLen)).map(padding)
        # valid_set = valid_set.map(lambda dataset: tokenizer(dataset["text"],truncation=True, max_length=config.MaxLen)).map(padding)
        
        # train_set= Dataset.from_dict({"id": train_set['train']['input_ids']}).with_format("torch")        
        # valid_set= Dataset.from_dict({"id": valid_set['train']['input_ids']}).with_format("torch")
        
        count=0
        TRAIN=[]
        train_loader=DataLoader(train_set,batch_size=50,shuffle=True)
        test_loader=DataLoader(valid_set,batch_size=100,shuffle=True)
       
        for i in train_loader:
            TRAIN.append(random.choice(i['id']))
            if count>100:
                break
            count+=1
        
        TRAIN=torch.stack(TRAIN,0) 
        optim=torch.optim.AdamW(model.parameters(),lr=1e-3)
        loss_fn=torch.nn.CrossEntropyLoss()
        loss_stats = {
            'train_loss': [],
            'test_loss': [],
            'train_acc': [],
            'test_acc': []
        }
        for epoch in tqdm(range(config.epochs)):
            for batch in tqdm(train_loader):
                optim.zero_grad()
        #         print(batch['id'][:,:-1])
                input_ids=batch['id'][:,:-1].to(device)
        #         print(input_ids.shape())
                labels=batch['id'][:,1:].to(device)

                labels=nn.functional.one_hot(labels,num_classes=50304).type(torch.float)
                # outputs=model.forward(input_ids).logits
                logits, loss = model.forward(input_ids)
                loss=loss_fn(logits, labels)
                loss.backward()
                optim.step()

            with torch.no_grad():
                # TextGenerator(40, start_tokens).on_epoch_end(epoch);
                train_input=TRAIN[:,:-1].to(device)
                train_output=TRAIN[:,1:].to(device)
                labels=nn.functional.one_hot(train_output,num_classes=50304).type(torch.float)
                # train_outputs=model(train_input).logits
                train_outputs, train_loss = model(train_input)
                train_loss=loss_fn(train_outputs,labels).cpu().item()
                loss_stats['train_loss'].append(train_loss)

                train_pred_softmax = torch.log_softmax(train_outputs, dim = 2)
                _, train_pred_tags = torch.max(train_pred_softmax, dim = 2)    
                train_correct_pred = (train_pred_tags == train_output).float()
                size=train_correct_pred.shape[0]*train_correct_pred.shape[1]
                train_acc = train_correct_pred.sum() / size
                train_acc = train_acc * 100
                train_acc=train_acc.cpu()
                loss_stats['train_acc'].append(train_acc.cpu())

                for b in tqdm(test_loader):
                    test_input=b['id'][:,:-1].to(device)
            #         print(input_ids.shape())
                    test_output=b['id'][:,1:].to(device)
                    labels=nn.functional.one_hot(test_output,num_classes=50304).type(torch.float)

                    # test_outputs=model.forward(test_input).logits
                    test_outputs, test_loss = model.forward(test_input)
                    # test_loss=loss_fn(test_outputs,labels).cpu().item()
                    test_pred_softmax = torch.log_softmax(test_outputs, dim = 2)
                    _, test_pred_tags = torch.max(test_pred_softmax, dim = 2)    
                    test_correct_pred = (test_pred_tags == test_output).float()
                    size=test_correct_pred.shape[0]*test_correct_pred.shape[1]
                    test_acc = test_correct_pred.sum() / size
                    test_acc = test_acc * 100
                    test_acc=test_acc.cpu()
                    loss_stats['test_acc'].append(test_acc.cpu())
                    loss_stats['test_loss'].append(test_loss)
            print(loss_stats)
                # wandb.log({"Train Loss":train_loss, "Test Loss":test_loss,"Train Accuracy":train_acc,"Test Accuracy":test_acc}, step=epoch)
        # TextGenerator(5, start_tokens,config.MaxLen,model).on_epoch_end(epoch);
        torch.save(model.state_dict(),f"{config.saveFolder}/transformer_weights.pth")


In [ ]:
training(config, model)

100%|██████████| 414/414 [00:11<00:00, 36.63it/s]

100%|██████████| 1/1 [00:13<00:00, 13.66s/it]

{'train_loss': [0.00024310272419825196], 'test_loss': [None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None], 'train_acc': [tensor(62.1775)], 'test_acc': [tensor(63.1579), tensor(62.5263), tensor(62.6842), tensor(63.0526), tensor(62.5789), tensor(62.6842), tensor(63.4737), tensor(62.6842), tensor(63.2632), tensor(62.3158), tensor(61.8947), tensor(62.3684), tensor(63.0526), tensor(61.6842), tensor(62.5263), tensor(62.8947), tensor(62.7895), tensor(62.6842), tensor(61.6316), tensor(62.0000), tensor(61.6842), tensor(62.2632), tensor(61.9474), tensor(63.7368), tensor(63.0526), tensor(61.5263), tensor(62.4737), tensor(62.6842), tensor(62.8421), tensor(62.7368), tensor(62.3684), tensor(63.0526), tensor(62.9474), tensor(62.8421), tensor(62

In [ ]:
start_prompt = valid_set[:3]
print(start_prompt)
# start_tokens=tokenizer(start_prompt)['input_ids']
num_tokens_generated = 20
text_gen_callback = TextGenerator(num_tokens_generated, start_tokens, model)

In [ ]:
text_gen_callback.on_epoch_end(0)

AttributeError: ignored